# Advanced FastAPI Features - Data Managmenet and Custom Headers

## Overview
In this notebook, we will explore advanced features of **FastAPI** such as handling query parameters, working with file uploads, setting custom headers, and managing cookies. We'll also cover how to handle form data, validate input using Pydantic models, and perform date-based calculations. By the end of this notebook, you'll have a solid understanding of how to build more complex APIs with FastAPI.


## Table of Contents
1. **Query Parameters and Optional Date Filtering**
2. **Custom Headers and Cookies**
3. **File Uploads and Form Handling**
4. **Pydantic Models for Data Validation**
5. **Assignment: Calculating Days Until Next Birthday**

## 1. Query Parameters and Optional Date Filtering

### Definition:
**Query Parameters** are key-value pairs appended to the URL after a `?` symbol. They allow clients to pass additional information to the server without modifying the path. In FastAPI, query parameters can be optional or required, and they can be validated using Python's type hints and optional constraints like `min_length`, `max_length`, and default values.

### Example 1: Fetching Event Details with Optional Date Filtering

```python
from fastapi import FastAPI, HTTPException
from datetime import date
from typing import Optional

app = FastAPI()

# Initialize a list to represent a fake database of events.
events_db = [
    {"event_id": 1, "name": "Tech Conference", "date": "2024-12-01"},
    {"event_id": 2, "name": "Movie Premiere", "date": "2024-12-15"},
    {"event_id": 3, "name": "Musical Festival", "date": "2024-12-20"}
]

@app.get("/events/{event_id}")
async def read_event(event_id: int, start_date: Optional[date] = None):
    """
    This endpoint fetches details of a specific event based on its ID and optionally filters by a start date.
    - `event_id`: A mandatory path parameter specifying the event ID.
    - `start_date`: An optional query parameter specifying the minimum date for filtering events.
    """
    # Find the event in the 'events_db' list by matching the 'event_id'.
    event = next((event for event in events_db if event["event_id"] == event_id), None)
    
    # If the event is not found, raise an HTTP 404 error.
    if not event:
        raise HTTPException(status_code=404, detail="Event not found")
    
    # If 'start_date' is provided, compare it with the event's date.
    if start_date and date.fromisoformat(event["date"]) < start_date:
        return {"message": "No events on or after the start date."}
    
    return event
```

#### Explanation:
- **Route**: `/events/{event_id}`
- **Parameters**:
  - `event_id`: A mandatory path parameter that identifies the event.
  - `start_date`: An optional query parameter that filters events based on their date.
- **Functionality**:
  - The function searches for an event in the `events_db` list by matching the `event_id`.
  - If no event is found, it raises an HTTP 404 error.
  - If `start_date` is provided, it checks whether the event's date is on or after the `start_date`. If not, it returns a message indicating no events match the criteria.

## 2. Custom Headers and Cookies

### Definition:
**Custom Headers** allow you to add additional metadata to the HTTP response, while **Cookies** store small pieces of data on the client side. In FastAPI, you can set custom headers using the `Response` object and retrieve cookies from the `Request` object. This is useful for implementing features like user preferences, authentication tokens, or tracking user behavior.

### Example 2: Fetching Items with User Preferences via Cookies

```python
from fastapi import FastAPI, Response, Request

app = FastAPI()

# Initialize a list to represent a mock database of items.
items = [
    {"id": 1, "name": "item1", "category": "Books"},
    {"id": 2, "name": "item2", "category": "Electronics"},
    {"id": 3, "name": "item3", "category": "Clothing"}
]

@app.get("/items/")
async def read_items(request: Request, response: Response):
    """
    This endpoint fetches items based on the user's preference stored in cookies.
    - `request`: Used to retrieve cookies from the client.
    - `response`: Used to set custom headers in the response.
    """
    # Retrieve the user's preference from the request cookies.
    user_preference = request.cookies.get('preference')
    
    # Set a custom header in the response.
    response.headers["Custom-Header"] = "Value"
    
    # If the user has a preference, filter the items by the user's preferred category.
    if user_preference:
        filtered_items = [item for item in items if item["category"] == user_preference]
        return {"items": filtered_items}
    
    # If no preference is specified, return all items.
    return {"items": items}
```

#### Explanation:
- **Route**: `/items/`
- **Parameters**:
  - `request`: Used to retrieve cookies from the client.
  - `response`: Used to set custom headers in the response.
- **Functionality**:
  - The function retrieves the user's preference from the `preference` cookie.
  - If a preference exists, it filters the `items` list by the user's preferred category.
  - A custom header `"Custom-Header"` is added to the response.
  - If no preference is found, all items are returned.

## 3. File Uploads and Form Handling

### Definition:
**File Uploads** allow users to send files to the server, while **Form Handling** processes form data submitted via HTML forms. In FastAPI, you can handle file uploads using the `UploadFile` class and form data using the `Form` class. This is commonly used in applications that require file processing, such as image uploads, document submissions, or form submissions.

### Example 3: Handling Form Uploads with File and Notes

```python
from fastapi import FastAPI, File, UploadFile, Form 

app = FastAPI()

@app.post("/submit-form")
async def handle_form_upload(file: UploadFile = File(...), notes: str = Form(...)):
    """
    This endpoint handles form uploads containing a file and additional notes.
    - `file`: The uploaded file (required).
    - `notes`: Additional notes provided in the form (required).
    """
    # Read the contents of the uploaded file asynchronously.
    contents = await file.read()
    
    # Count the number of lines in the file's contents.
    num_lines = contents.decode("utf-8").count("\n") + 1
    
    # Calculate the file size by counting the number of bytes in 'contents'.
    file_size = len(contents)
    
    # Reset the file's pointer to the beginning after reading.
    await file.seek(0)
    
    # Return a JSON response containing details about the uploaded file and the notes.
    return {
        "file_name": file.filename,
        "file_size": file_size,
        "number_of_lines": num_lines,
        "notes": notes
    }
```

#### Explanation:
- **Route**: `/submit-form`
- **Parameters**:
  - `file`: The uploaded file, which is required.
  - `notes`: Additional notes provided in the form, which are also required.
- **Functionality**:
  - The function reads the contents of the uploaded file asynchronously.
  - It calculates the number of lines and the file size.
  - The file pointer is reset to the beginning after reading.
  - The function returns a JSON response with details about the file and the notes.

## 4. Pydantic Models for Data Validation

### Definition:
**Pydantic Models** are used for data validation and serialization in FastAPI. They allow you to define the structure of incoming and outgoing data, enforce type constraints, and apply additional validation rules using fields like `Field`, `min_length`, `max_length`, and custom patterns. Pydantic ensures that the data conforms to the expected format before it is processed by your application.

### Example 4: Creating and Searching Books Using Pydantic Models

```python
from fastapi import FastAPI, Query 
from typing import Optional
from datetime import date
from pydantic import BaseModel

app = FastAPI()

class Book(BaseModel):
    title: str
    price: float
    publication_date: Optional[date] = None

@app.post("/books/")
async def create_book(book: Book):
    """
    This endpoint creates a new book using a Pydantic model for validation.
    - `book`: A Pydantic model representing the book's data.
    """
    return {"message": "Book created Successfully", "book": book}

@app.get("/books/")
async def read_books(q: Optional[str] = Query(None, min_length=3, max_length=50)):
    """
    This endpoint allows users to search for books based on a query string.
    - `q`: An optional query parameter for searching books by title.
    """
    fake_db = [{"title": "Book 1"}, {"title": "Book 2"}]
    
    if q:
        result = filter(lambda book: q.lower() in book["title"].lower(), fake_db)
        return {"search": q, "result": list(result)}
    
    return {"books": fake_db}
```

#### Explanation:
- **Pydantic Model**: `Book` defines the structure of the incoming data with fields for `title`, `price`, and `publication_date`.
- **Routes**:
  - `/books/` (POST): Creates a new book using the `Book` model for validation.
  - `/books/` (GET): Searches for books based on an optional query parameter `q`.
- **Functionality**:
  - The POST route validates the incoming book data and returns a success message.
  - The GET route allows users to search for books by title using the `q` parameter.


## 5. Assignment: Calculating Days Until Next Birthday

### Definition:
This assignment involves creating an API endpoint that calculates the number of days until the next birthday based on a given date of birth. The endpoint should handle invalid date formats gracefully and return the correct number of days until the next birthday.

### Example 5: Calculating Days Until Next Birthday

```python
from fastapi import FastAPI, HTTPException
from datetime import datetime

app = FastAPI()

@app.get("/birthday/{date_of_birth}")
async def calculate_days(date_of_birth: str):
    """
    This endpoint calculates the number of days until the next birthday.
    - `date_of_birth`: A mandatory path parameter specifying the user's date of birth in YYYY-MM-DD format.
    """
    try:
        # Attempt to convert the 'date_of_birth' string to a datetime object.
        dob = datetime.fromisoformat(date_of_birth).date()
    except ValueError:
        # If the date format is incorrect, raise an HTTP 400 error with a descriptive message.
        raise HTTPException(status_code=400, detail="Invalid date format. Please use YYYY-MM-DD")
    
    # Get the current date.
    today = datetime.now().date()
    
    # Calculate the date of the next birthday.
    next_birthday = dob.replace(year=today.year)
    if today > next_birthday:
        next_birthday = dob.replace(year=today.year + 1)
    
    # Calculate the number of days until the next birthday.
    days_until = (next_birthday - today).days
    
    # Return the number of days in a JSON response.
    return {"days_until_next_birthday": days_until}
```

#### Explanation:
- **Route**: `/birthday/{date_of_birth}`
- **Parameters**:
  - `date_of_birth`: A mandatory path parameter specifying the user's date of birth in `YYYY-MM-DD` format.
- **Functionality**:
  - The function converts the `date_of_birth` string to a `datetime.date` object.
  - It calculates the next birthday by comparing the current date with the user's birth month and day.
  - If the current date is past the user's birthday this year, it calculates the next year's birthday.
  - Finally, it returns the number of days until the next birthday.